<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 将从头实现的GPT结构转换为Llama2
- 本notebook一步步将原始GPT架构转换为Llama2；注意，GPT和GPT2架构相同
- 为什么不是Llama1或Llama3？
  - Llama1架构和Llama2相似，除了Llama2有更大的上下文窗口；Llama1的权重访问不方便并且有很多限制，因此将目标设置为Llama2更合适
  - 关于Llama 3，会提供一个单独的笔记本，将Llama 2转换为Llama 3（只有几个小的额外更改）
- 本notebook中的解释被有意识地保持在最小限度，以避免不必要的臃肿，并专注于主要代码
- 欲了解更多信息，请参阅Llama 2论文：[Llama 2: Open Foundation and Fine-Tuned Chat Models (2023)](https://arxiv.org/abs/2307.09288)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/gpt2-to-llama2-llama3.webp?1">

- 本notebook中需要使用的库

In [ ]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",  # 下载预训练权重
    "sentencepiece",    # 实现分词器
    "torch",            # 实现模型
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

&nbsp;
# 1. 逐步转换GPT模型实现

- 在本节中，将通过[第4章](../../ch04/01_main-chapter-code/ch04.ipynb)中的GPT模型代码逐步修改它以实现Llama2架构
- 之后，将加载Meta AI共享的原始Llama2权重


&nbsp;
## 1.1 用RMSNorm层替换LayerNorm层

- 首先，用均方根层归一化（RMSNorm）替换LayerNorm
- LayerNorm使用均值和方差归一化输入，而RMSNorm仅使用均方根，这提高了计算效率
- RMSNorm操作如下，其中$x$是输入，$\gamma$是可训练参数（向量），$\epsilon$是一个小常数，用于避免零除错误：

$$y_i = \frac{x_i}{\text{RMS}(x)} \gamma_i, \quad \text{其中} \quad \text{RMS}(x) = \sqrt{\epsilon + \frac{1}{n} \sum x_i^2}$$

- 更多详情，请参阅论文[Root Mean Square Layer Normalization (2019)](https://arxiv.org/abs/1910.07467)


In [2]:
import torch
import torch.nn as nn

In [4]:
#####################################
# Chapter 4
#####################################

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim: int) -> None:
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift


class RMSNorm(nn.Module):
    def __init__(self, emb_dim: int, eps: float = 1e-5) -> None:
        super().__init__()
        self.eps = eps
        self.emb_dim = emb_dim
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        means = x.pow(2).mean(dim=-1, keepdim=True)
        x_normed = x * torch.rsqrt(means + self.eps)
        return (x_normed * self.weight).to(dtype=x.dtype)

- 以下代码cell校验上述实现和torch内置实现是否相同

In [ ]:
torch.manual_seed(123)

example_batch = torch.randn(2, 3, 4)  # 随机生成一个2x3x4的tensor

rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])  # 创建RMSNorm实例
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)  # 创建torch.nn.RMSNorm实例

assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))  # 校验两个实例的输出是否相同

&nbsp;
## 1.2 用SiLU激活函数替换GELU激活函数

- Llama使用SiLU激活函数（而不是GELU），它也被称为Swish函数：

$$
\text{silu}(x) = x \cdot \sigma(x), \quad \text{其中} \quad \sigma(x) \text{ 是logistic sigmoid函数。}
$$

- 有关更多信息，请参阅SiLU论文：[Sigmoid-Weighted Linear Units for Neural Network Function Approximation in Reinforcement Learning (2017)](https://arxiv.org/abs/1702.03118)

In [5]:
#####################################
# Chapter 4
#####################################

# class GELU(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         return 0.5 * x * (1 + torch.tanh(
#             torch.sqrt(torch.tensor(2.0 / torch.pi)) *
#             (x + 0.044715 * torch.pow(x, 3))
#         ))


class SiLU(nn.Module):
    def __init__(self) -> None:
        super(SiLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * torch.sigmoid(x)

In [6]:
silu = SiLU()

assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

&nbsp;
## 1.3 更新FeedForward层

- 实际上，Llama使用了一种称为SwiGLU的SiLU的"门控线性单元"(GLU)变体，这基本上导致了一个结构略有不同的`FeedForward`模块
- SwiGLU在前馈层中使用门控机制，公式为：

$$\text{SwiGLU}(x) = \text{SiLU}(\text{Linear}_1(x)) * \text{Linear}_2(x)$$

- 这里，$\text{Linear}_1$和$\text{Linear}_2$是两个线性层，$*$表示元素级乘法
- 第三个线性层$\text{Linear}_3$在这个门控激活之后应用

- 有关更多信息，请参阅SwiGLU论文：[GLU Variants Improve Transformer (2020)](https://arxiv.org/abs/2002.05202)


In [ ]:
#####################################
# Chapter 4
#####################################
# class FeedForward(nn.Module):
#     def __init__(self, cfg) -> None:
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
#             GELU(),
#             nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
#         )

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         return self.layers(x)

In [7]:
class FeedForward(nn.Module):
    def __init__(self, cfg) -> None:
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        x = self.silu(x_fc1) * x_fc2
        return self.fc3(x)

&nbsp;
## 1.4 实现RoPE

- 在GPT模型中，位置嵌入是如下实现的：

```python
self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
```

- 与传统的绝对位置嵌入不同，Llama使用旋转位置嵌入(RoPE)，这使它能够同时捕获绝对和相对位置信息
- RoPE的参考论文是[RoFormer: Enhanced Transformer with Rotary Position Embedding (2021)](https://arxiv.org/abs/2104.09864)
- RoPE有两种等价的实现方式：分半版本和交错偶数/奇数版本；只要一致地配对维度并使用相同的余弦/正弦顺序，它们在数学上是相同的（有关更多信息，请参阅此[GitHub讨论](https://github.com/rasbt/LLMs-from-scratch/issues/751)）
- 这段代码使用了与Hugging Face transformers（[modeling_llama.py](https://github.com/huggingface/transformers/blob/e42587f596181396e1c4b63660abf0c736b10dae/src/transformers/models/llama/modeling_llama.py#L173-L188)）类似的RoPE分半方法。
- 然而，原始的RoPE论文和Meta的官方Llama 2仓库使用交错（偶数/奇数）版本（[llama/model.py](https://github.com/meta-llama/llama/blob/6c7fe276574e78057f917549435a2554000a876d/llama/model.py#L64-L74)）；但如前所述，它们是等价的

In [ ]:
def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096):
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # 计算逆频率
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # 生成位置索引
    positions = torch.arange(context_length)

    # 计算角度
    angles = positions.unsqueeze(1) * inv_freq.unsqueeze(0)  # Shape: (context_length, head_dim // 2)

    # 拓展角度适配head_dim
    angles = torch.cat([angles, angles], dim=1)  # Shape: (context_length, head_dim)

    # 预计算sine和cosine
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin

def compute_rope(x, cos, sin):
    # x: (batch_size, num_heads, seq_len, head_dim)
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 == 0, "Head dimension must be even"

    # 将x分割为前后两部分
    x1 = x[..., : head_dim // 2]  # First half
    x2 = x[..., head_dim // 2 :]  # Second half

    # 只取序列长度
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, seq_len, head_dim)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # 应用旋转变换
    rotated = torch.cat((-x2, x1), dim=-1)
    x_rotated = (x * cos) + (rotated * sin)

    return x_rotated.to(dtype=x.dtype)

- 以下是将RoPE应用于`q`和`k`张量的示例

In [ ]:
# Settings
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

# 初始化RoPE参数
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# 随机创建queries和keys
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.randn(batch_size, num_heads, context_len, head_dim)

# 应用旋转位置编码
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)

&nbsp;
## 1.5 在MultiHeadAttention模块中添加RoPE

- 需要注意的是，GPT将位置嵌入应用于输入，而Llama在自注意力机制本身中对查询向量和键向量进行旋转、
- 在这里，使用适当的RoPE代码修改`MultiHeadAttention`类
- 此外，移除了`qkv_bias`选项，并硬编码了`bias=False`设置
- 此外，添加了一个dtype设置，以便之后能够以较低精度实例化模型
- 提示：由于`TransformerBlock`（在下一节中）完全重复，可以简化代码，并且只需为每个`MultiHeadAttention`模块初始化缓冲区一次；然而，将预计算的`RoPE`参数添加到`MultiHeadAttention`类中，以便它能够作为一个独立的模块运行

In [ ]:
#####################################
# Chapter 3
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):  # ,dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        ################################### NEW ###################################
        # Set bias=False and dtype=dtype for all linear layers below
        ###########################################################################
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)  # Linear layer to combine head outputs
        # self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
        ###########################################################################


    def forward(self, x):

        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        ################################### NEW ###################################
        keys = compute_rope(keys, self.cos, self.sin)
        queries = compute_rope(queries, self.cos, self.sin)
        ###########################################################################

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

&nbsp;
## 1.6 更新TransformerBlock模块

- 在这个阶段，大部分艰苦的工作已经完成；现在可以更新`TransformerBlock`以使用上面实现的代码
- 这意味着
- 用RMSNorm替换LayerNorm
- 移除dropout
- 移除`qkv_bias`设置
- 添加`dtype`设置

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dtype=cfg["dtype"]  # NEW
            # dropout=cfg["drop_rate"],
            # qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)

        ################################### NEW ###################################
        # self.norm1 = LayerNorm(cfg["emb_dim"])
        # self.norm2 = LayerNorm(cfg["emb_dim"])
        self.norm1 = RMSNorm(cfg["emb_dim"])
        self.norm2 = RMSNorm(cfg["emb_dim"])
        ###########################################################################

        # self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

&nbsp;
## 1.7 更新model类

- 还记得第五章中， `TransformerBlock`是主模型中的一个重复模块吗
- `Llama`模型几乎完成了；只需要更新围绕`TransformerBlock`的模型代码
- 这意味着
- 移除绝对位置嵌入，因为现在有RoPE嵌入
- 用RMSNorm替换LayerNorm
- 添加dtype设置

In [ ]:
# class GPTModel(nn.Module):
class Llama2Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
        # self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        ################################### NEW ###################################
        # self.final_norm = LayerNorm(cfg["emb_dim"])
        self.final_norm = RMSNorm(cfg["emb_dim"])
        ###########################################################################
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

    def forward(self, in_idx):
        # batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds  # + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits